In [1]:
import polars as rs

In [2]:
columns = ["user", "poi", "date", "TZ"]

In [3]:
data = rs.read_csv(
    "dataset_TIST2015/dataset_TIST2015_Checkins.txt",
    has_header=False,
    low_memory=True,
    separator="\t",
)

In [4]:
data.columns = columns

In [5]:
data

user,poi,date,TZ
i64,str,str,i64
50756,"""4f5e3a72e4b053…","""Tue Apr 03 18:…",240
190571,"""4b4b87b5f964a5…","""Tue Apr 03 18:…",180
221021,"""4a85b1b3f964a5…","""Tue Apr 03 18:…",-240
66981,"""4b4606f2f964a5…","""Tue Apr 03 18:…",-300
21010,"""4c2b4e8a9a559c…","""Tue Apr 03 18:…",240
…,…,…,…
16349,"""4c957755c8a1bf…","""Mon Sep 16 23:…",-240
256757,"""4c8bbb6d9ef022…","""Mon Sep 16 23:…",-180
66425,"""513e82a5e4b0ed…","""Mon Sep 16 23:…",-180


In [6]:
data_users = (
    data.lazy()
    .group_by("user")
    .agg(
        [
            rs.col("poi").n_unique().alias("n_pois"),
            rs.col("poi").count().alias("n_checkins"),
            # turn the rest into a list
            rs.col("poi").alias("pois"),
            rs.col("date").alias("dates"),
            rs.col("TZ").alias("TZs"),
        ]
    )
).collect()

In [7]:
data_users.describe()

statistic,user,n_pois,n_checkins,pois,dates,TZs
str,f64,f64,f64,f64,f64,f64
"""count""",266909.0,266909.0,266909.0,266909.0,266909.0,266909.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",133455.0,56.477459,124.62537,null,null,null
"""std""",77050.135837,45.968603,140.692138,null,null,null
"""min""",1.0,1.0,1.0,null,null,null
"""25%""",66728.0,30.0,61.0,null,null,null
"""50%""",133455.0,49.0,93.0,null,null,null
"""75%""",200182.0,71.0,148.0,null,null,null
"""max""",266909.0,1246.0,5430.0,null,null,null


In [8]:
data_culled = data_users.filter(
    (rs.col("n_checkins") > 20) & (rs.col("n_checkins") < 50)
).drop_nulls()

In [9]:
del data
del data_users

import gc

gc.collect()

142

In [10]:
# extract unique elements from each lists in data_culled["pois"]
out = data_culled.with_columns(
    [
        rs.col("pois").list.unique(),
        rs.col("pois").list.unique().list.len().alias("n_unique_pois"),
    ]
)

In [11]:
data_culled

user,n_pois,n_checkins,pois,dates,TZs
i64,u32,u32,list[str],list[str],list[i64]
250774,25,26,"[""501655d7e4b0c79e4914ad13"", ""4d596f261d4a8cfaaf6b8ced"", … ""4c3efdc08d99b713a3021b06""]","[""Tue Nov 27 21:12:14 +0000 2012"", ""Sun Dec 09 10:22:54 +0000 2012"", … ""Sat Aug 17 11:46:58 +0000 2013""]","[120, 120, … 180]"
248598,30,46,"[""4e38456dd164374eb68f388c"", ""4d0d1f5e1f6bf04d1c9d7b31"", … ""4e9f06d19a52cd825d216b31""]","[""Sat Apr 07 14:07:10 +0000 2012"", ""Sun Apr 08 12:31:15 +0000 2012"", … ""Tue Jul 02 13:55:38 +0000 2013""]","[180, 180, … 180]"
236736,31,39,"[""4b8a9821f964a520187432e3"", ""4c42710b3735be9ac71d1aa4"", … ""4bf2c93620960f47bba66e2f""]","[""Sat Dec 29 20:57:43 +0000 2012"", ""Sat Dec 29 23:47:48 +0000 2012"", … ""Wed Sep 04 14:06:49 +0000 2013""]","[-300, -300, … -240]"
61240,24,25,"[""4f4f47f3e4b066791aaa6380"", ""4e2463e7b0fbdf9ba7c6d470"", … ""51a32447498ea34bef99f9f4""]","[""Tue Feb 19 07:55:08 +0000 2013"", ""Mon Mar 04 09:14:47 +0000 2013"", … ""Sat Aug 10 21:22:48 +0000 2013""]","[120, 120, … 180]"
46160,24,26,"[""4e58ea2a18388cd5cb9b6be5"", ""4daa809493a04642f081ebf4"", … ""509d066ae4b057cac320cf6a""]","[""Sat Jun 30 20:27:11 +0000 2012"", ""Sat Jun 30 23:44:46 +0000 2012"", … ""Mon Apr 22 18:06:33 +0000 2013""]","[-360, -360, … -360]"
…,…,…,…,…,…
75855,34,39,"[""4c10bacbb93cc9b66c2357e0"", ""4e69c76dd22d0e4cf5c0138f"", … ""4ebaaafb6c25faf0a4f28a04""]","[""Sun Apr 08 05:21:17 +0000 2012"", ""Sun Apr 08 05:22:01 +0000 2012"", … ""Mon Aug 26 10:43:43 +0000 2013""]","[330, 330, … 330]"
108915,35,42,"[""4bf7409c5efe2d7f51c56834"", ""4283ee00f964a52089221fe3"", … ""49baa706f964a52091531fe3""]","[""Sat Apr 21 03:11:51 +0000 2012"", ""Sun May 13 00:35:53 +0000 2012"", … ""Sat Aug 31 02:35:30 +0000 2013""]","[-420, -420, … -420]"
150951,31,41,"[""4b99c918f964a520b49135e3"", ""4c91d3fb0c209c74684734ff"", … ""4e780b81c65b409c469c314f""]","[""Thu Apr 12 07:53:28 +0000 2012"", ""Thu Apr 19 11:13:08 +0000 2012"", … ""Tue Apr 02 22:52:23 +0000 2013""]","[420, 420, … 420]"


In [12]:
out

user,n_pois,n_checkins,pois,dates,TZs,n_unique_pois
i64,u32,u32,list[str],list[str],list[i64],u32
250774,25,26,"[""4e14bced7d8b4c0d631165d5"", ""4d889ba5b799a35df5c88463"", … ""515e9ae3e4b0e31714095179""]","[""Tue Nov 27 21:12:14 +0000 2012"", ""Sun Dec 09 10:22:54 +0000 2012"", … ""Sat Aug 17 11:46:58 +0000 2013""]","[120, 120, … 180]",25
248598,30,46,"[""4f7130abe4b0c719b22f91e6"", ""4cbc6313adcd5481dc122ea2"", … ""51399fffe4b0b84240654717""]","[""Sat Apr 07 14:07:10 +0000 2012"", ""Sun Apr 08 12:31:15 +0000 2012"", … ""Tue Jul 02 13:55:38 +0000 2013""]","[180, 180, … 180]",30
236736,31,39,"[""4b494bdaf964a520bb6c26e3"", ""4bc910e9511f95215e1badc7"", … ""4bc73ba38b7c9c74e50236cf""]","[""Sat Dec 29 20:57:43 +0000 2012"", ""Sat Dec 29 23:47:48 +0000 2012"", … ""Wed Sep 04 14:06:49 +0000 2013""]","[-300, -300, … -240]",31
61240,24,25,"[""4b6d43b6f964a520386e2ce3"", ""4b744f96f964a52053d42de3"", … ""4b60500bf964a5200edf29e3""]","[""Tue Feb 19 07:55:08 +0000 2013"", ""Mon Mar 04 09:14:47 +0000 2013"", … ""Sat Aug 10 21:22:48 +0000 2013""]","[120, 120, … 180]",24
46160,24,26,"[""4daa809493a04642f081ebf4"", ""4cc31383be40a35d6dbc754c"", … ""4d279c2655a8b60c6956c5c0""]","[""Sat Jun 30 20:27:11 +0000 2012"", ""Sat Jun 30 23:44:46 +0000 2012"", … ""Mon Apr 22 18:06:33 +0000 2013""]","[-360, -360, … -360]",24
…,…,…,…,…,…,…
75855,34,39,"[""4b913c94f964a5202fac33e3"", ""4c10bacbb93cc9b66c2357e0"", … ""4dc3c68efa76d685cdb51d65""]","[""Sun Apr 08 05:21:17 +0000 2012"", ""Sun Apr 08 05:22:01 +0000 2012"", … ""Mon Aug 26 10:43:43 +0000 2013""]","[330, 330, … 330]",34
108915,35,42,"[""40a2ba80f964a5200ef31ee3"", ""4a332eb4f964a520109b1fe3"", … ""4cc639ce8062468878fa402f""]","[""Sat Apr 21 03:11:51 +0000 2012"", ""Sun May 13 00:35:53 +0000 2012"", … ""Sat Aug 31 02:35:30 +0000 2013""]","[-420, -420, … -420]",35
150951,31,41,"[""4e510e7fd22daf51d28bedfb"", ""4bdbe47663c5c9b65ef02968"", … ""4bb92f18cf2fc9b6b157a002""]","[""Thu Apr 12 07:53:28 +0000 2012"", ""Thu Apr 19 11:13:08 +0000 2012"", … ""Tue Apr 02 22:52:23 +0000 2013""]","[420, 420, … 420]",31


In [13]:
l = out["pois"][0].to_list()

In [14]:
len(set(l))

25

In [15]:
l2 = data_culled["pois"][0].to_list()

In [16]:
len(l2)

26

In [17]:
len(set(l2))

25

In [18]:
out

user,n_pois,n_checkins,pois,dates,TZs,n_unique_pois
i64,u32,u32,list[str],list[str],list[i64],u32
250774,25,26,"[""4e14bced7d8b4c0d631165d5"", ""4d889ba5b799a35df5c88463"", … ""515e9ae3e4b0e31714095179""]","[""Tue Nov 27 21:12:14 +0000 2012"", ""Sun Dec 09 10:22:54 +0000 2012"", … ""Sat Aug 17 11:46:58 +0000 2013""]","[120, 120, … 180]",25
248598,30,46,"[""4f7130abe4b0c719b22f91e6"", ""4cbc6313adcd5481dc122ea2"", … ""51399fffe4b0b84240654717""]","[""Sat Apr 07 14:07:10 +0000 2012"", ""Sun Apr 08 12:31:15 +0000 2012"", … ""Tue Jul 02 13:55:38 +0000 2013""]","[180, 180, … 180]",30
236736,31,39,"[""4b494bdaf964a520bb6c26e3"", ""4bc910e9511f95215e1badc7"", … ""4bc73ba38b7c9c74e50236cf""]","[""Sat Dec 29 20:57:43 +0000 2012"", ""Sat Dec 29 23:47:48 +0000 2012"", … ""Wed Sep 04 14:06:49 +0000 2013""]","[-300, -300, … -240]",31
61240,24,25,"[""4b6d43b6f964a520386e2ce3"", ""4b744f96f964a52053d42de3"", … ""4b60500bf964a5200edf29e3""]","[""Tue Feb 19 07:55:08 +0000 2013"", ""Mon Mar 04 09:14:47 +0000 2013"", … ""Sat Aug 10 21:22:48 +0000 2013""]","[120, 120, … 180]",24
46160,24,26,"[""4daa809493a04642f081ebf4"", ""4cc31383be40a35d6dbc754c"", … ""4d279c2655a8b60c6956c5c0""]","[""Sat Jun 30 20:27:11 +0000 2012"", ""Sat Jun 30 23:44:46 +0000 2012"", … ""Mon Apr 22 18:06:33 +0000 2013""]","[-360, -360, … -360]",24
…,…,…,…,…,…,…
75855,34,39,"[""4b913c94f964a5202fac33e3"", ""4c10bacbb93cc9b66c2357e0"", … ""4dc3c68efa76d685cdb51d65""]","[""Sun Apr 08 05:21:17 +0000 2012"", ""Sun Apr 08 05:22:01 +0000 2012"", … ""Mon Aug 26 10:43:43 +0000 2013""]","[330, 330, … 330]",34
108915,35,42,"[""40a2ba80f964a5200ef31ee3"", ""4a332eb4f964a520109b1fe3"", … ""4cc639ce8062468878fa402f""]","[""Sat Apr 21 03:11:51 +0000 2012"", ""Sun May 13 00:35:53 +0000 2012"", … ""Sat Aug 31 02:35:30 +0000 2013""]","[-420, -420, … -420]",35
150951,31,41,"[""4e510e7fd22daf51d28bedfb"", ""4bdbe47663c5c9b65ef02968"", … ""4bb92f18cf2fc9b6b157a002""]","[""Thu Apr 12 07:53:28 +0000 2012"", ""Thu Apr 19 11:13:08 +0000 2012"", … ""Tue Apr 02 22:52:23 +0000 2013""]","[420, 420, … 420]",31


In [19]:
unique_pois = out["pois"]

In [20]:
frequent_pois = unique_pois.list.explode().value_counts().filter(rs.col("count") >= 10)

In [21]:
frequent_pois

pois,count
str,u32
"""4b424ed8f964a5…",10
"""4beff54cc8d920…",12
"""4b937e60f964a5…",10
"""4d445fdbbbb1a1…",12
"""4c60459cb6f3be…",36
…,…
"""4b1d716ef964a5…",15
"""4f6a3a10d5fbee…",10
"""44cf0ff8f964a5…",14


In [22]:
frequent_pois = frequent_pois["pois"]
frequent_pois = set(frequent_pois.to_list())

In [23]:
data_culled

user,n_pois,n_checkins,pois,dates,TZs
i64,u32,u32,list[str],list[str],list[i64]
250774,25,26,"[""501655d7e4b0c79e4914ad13"", ""4d596f261d4a8cfaaf6b8ced"", … ""4c3efdc08d99b713a3021b06""]","[""Tue Nov 27 21:12:14 +0000 2012"", ""Sun Dec 09 10:22:54 +0000 2012"", … ""Sat Aug 17 11:46:58 +0000 2013""]","[120, 120, … 180]"
248598,30,46,"[""4e38456dd164374eb68f388c"", ""4d0d1f5e1f6bf04d1c9d7b31"", … ""4e9f06d19a52cd825d216b31""]","[""Sat Apr 07 14:07:10 +0000 2012"", ""Sun Apr 08 12:31:15 +0000 2012"", … ""Tue Jul 02 13:55:38 +0000 2013""]","[180, 180, … 180]"
236736,31,39,"[""4b8a9821f964a520187432e3"", ""4c42710b3735be9ac71d1aa4"", … ""4bf2c93620960f47bba66e2f""]","[""Sat Dec 29 20:57:43 +0000 2012"", ""Sat Dec 29 23:47:48 +0000 2012"", … ""Wed Sep 04 14:06:49 +0000 2013""]","[-300, -300, … -240]"
61240,24,25,"[""4f4f47f3e4b066791aaa6380"", ""4e2463e7b0fbdf9ba7c6d470"", … ""51a32447498ea34bef99f9f4""]","[""Tue Feb 19 07:55:08 +0000 2013"", ""Mon Mar 04 09:14:47 +0000 2013"", … ""Sat Aug 10 21:22:48 +0000 2013""]","[120, 120, … 180]"
46160,24,26,"[""4e58ea2a18388cd5cb9b6be5"", ""4daa809493a04642f081ebf4"", … ""509d066ae4b057cac320cf6a""]","[""Sat Jun 30 20:27:11 +0000 2012"", ""Sat Jun 30 23:44:46 +0000 2012"", … ""Mon Apr 22 18:06:33 +0000 2013""]","[-360, -360, … -360]"
…,…,…,…,…,…
75855,34,39,"[""4c10bacbb93cc9b66c2357e0"", ""4e69c76dd22d0e4cf5c0138f"", … ""4ebaaafb6c25faf0a4f28a04""]","[""Sun Apr 08 05:21:17 +0000 2012"", ""Sun Apr 08 05:22:01 +0000 2012"", … ""Mon Aug 26 10:43:43 +0000 2013""]","[330, 330, … 330]"
108915,35,42,"[""4bf7409c5efe2d7f51c56834"", ""4283ee00f964a52089221fe3"", … ""49baa706f964a52091531fe3""]","[""Sat Apr 21 03:11:51 +0000 2012"", ""Sun May 13 00:35:53 +0000 2012"", … ""Sat Aug 31 02:35:30 +0000 2013""]","[-420, -420, … -420]"
150951,31,41,"[""4b99c918f964a520b49135e3"", ""4c91d3fb0c209c74684734ff"", … ""4e780b81c65b409c469c314f""]","[""Thu Apr 12 07:53:28 +0000 2012"", ""Thu Apr 19 11:13:08 +0000 2012"", … ""Tue Apr 02 22:52:23 +0000 2013""]","[420, 420, … 420]"


In [24]:
data_culled = data_culled.with_columns(
    [
        rs.col("pois")
        .list.eval(
            rs.element().is_in(frequent_pois),
        )
        .alias("is_frequent")
    ]
)  # prep mask

In [25]:
final_data = (
    data_culled.lazy()
    .with_row_index()
    .explode(
        [
            "pois",
            "dates",
            "TZs",
            "is_frequent",
        ]
    )
    .group_by("index")
    .agg(
        [
            rs.col("pois").filter(rs.col("is_frequent")).alias("pois"),
            rs.col("dates").filter(rs.col("is_frequent")).alias("dates"),
            rs.col("TZs").filter(rs.col("is_frequent")).alias("TZs"),
            rs.col("pois").filter(rs.col("is_frequent")).n_unique().alias("n_pois"),
            rs.col("pois").filter(rs.col("is_frequent")).count().alias("n_checkins"),
        ]
    )
    .filter(rs.col("n_checkins") > 0)
    .filter(rs.col("n_pois") > 0)
    .collect()
)  # filter out infrequent pois and users with no pois

In [26]:
final_data.describe()

statistic,index,pois,dates,TZs,n_pois,n_checkins
str,f64,f64,f64,f64,f64,f64
"""count""",19862.0,19862.0,19862.0,19862.0,19862.0,19862.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,0.0
"""mean""",10859.864012,null,null,null,6.123452,8.831437
"""std""",6265.413406,null,null,null,4.609024,6.877662
"""min""",0.0,null,null,null,1.0,1.0
"""25%""",5441.0,null,null,null,3.0,4.0
"""50%""",10861.0,null,null,null,5.0,7.0
"""75%""",16276.0,null,null,null,8.0,12.0
"""max""",21696.0,null,null,null,32.0,46.0


In [27]:
import geohash2 as gh

pois = rs.read_csv(
    "dataset_TIST2015/dataset_TIST2015_POIs.txt",
    has_header=False,
    low_memory=True,
    separator="\t",
)
pois.columns = ["poi", "lat", "long", "category", "country"]
pois = pois.drop("category").drop("country")

In [28]:
pois = (
    pois.lazy()
    .filter(rs.col("poi").is_in(frequent_pois))
    .select(
        [
            rs.col("poi"),
            rs.struct(
                [
                    rs.col("lat").cast(rs.Float32),
                    rs.col("long").cast(rs.Float32),
                ]
            )
            .alias("location")
            .map_elements(
                lambda s: gh.encode(s["lat"], s["long"], precision=6),
                return_dtype=rs.String,
            )
            .alias("geohash"),
        ]
    )
    .collect()
)

In [29]:
poi_geo_dict = dict(zip(pois["poi"], pois["geohash"]))

In [30]:
# for each row in final_data, add the geohash of the pois


final_data = final_data.with_columns(
    [
        rs.col("pois")
        .map_elements(
            lambda s: [poi_geo_dict[s] for s in s],
        )
        .alias("geohashes")
    ]
)

/tmp/ipykernel_89916/3027446639.py:4: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  final_data = final_data.with_columns(


In [31]:
final_data["dates"][79].to_list()

['Tue May 29 18:14:16 +0000 2012',
 'Sat Sep 15 15:45:35 +0000 2012',
 'Sat Sep 15 22:23:37 +0000 2012',
 'Sun Nov 11 18:52:19 +0000 2012',
 'Fri Apr 05 13:21:20 +0000 2013',
 'Sat Apr 06 21:05:16 +0000 2013',
 'Sun Apr 07 13:54:32 +0000 2013']

In [32]:
final_data["TZs"][79].to_list()

[120, -240, -420, -480, -420, -240, -240]

In [104]:
import datetime


def UTC_to_local(utc, tz):

    date = datetime.datetime.strptime(utc, "%a %b %d %H:%M:%S %z %Y")
    date = date.replace(tzinfo=datetime.timezone.utc)

    # shift by tz offset

    date = date.astimezone(datetime.timezone(datetime.timedelta(minutes=tz)))

    date_s = datetime.datetime.strftime(date, "%Y-%m-%d %H:%M:%S")
    return date_s

In [105]:
UTC_to_local("Mon May 21 15:53:01 +0000 2012", -420)

'2012-05-21 08:53:01'

In [106]:
# get 5 rows of data
final_data_subset = final_data.head(100)

In [107]:
final_data_subset = final_data_subset.with_columns(
    [
        rs.struct([rs.col("dates"), rs.col("TZs")])
        .alias("times")
        .map_elements(
            lambda struct: [
                UTC_to_local(date, tz)
                for date, tz in zip(struct["dates"], struct["TZs"])
            ]
        )
    ]
)

/tmp/ipykernel_89916/1065629302.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  final_data_subset = final_data_subset.with_columns(


In [108]:
def to_UNIX_time(date):
    return datetime.datetime.strptime(date, "%Y-%m-%d %H:%M:%S").timestamp()

In [109]:
final_sorted_sub = final_data_subset.select(
    [
        rs.struct(
            [
                rs.col("pois"),
                rs.col("times"),
            ]
        ).map_elements(
            lambda struct: [
                poi
                for poi, _ in sorted(
                    zip(
                        struct["pois"], [to_UNIX_time(date) for date in struct["times"]]
                    ),
                    key=lambda s: s[1],
                )
            ]
        ),
        rs.struct(
            [
                rs.col("geohashes"),
                rs.col("times"),
            ]
        ).map_elements(
            lambda struct: [
                geo
                for geo, _ in sorted(
                    zip(
                        struct["geohashes"],
                        [to_UNIX_time(date) for date in struct["times"]],
                    ),
                    key=lambda s: s[1],
                )
            ]
        ),
        rs.col("times")
        .map_elements(lambda dates: sorted(dates, key=to_UNIX_time))
        .alias("times_sorted"),
        rs.col("n_checkins"),
    ]
)

sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
sys:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.


In [110]:
final_sorted_sub

pois,geohashes,times_sorted,n_checkins
list[str],list[str],list[str],u32
"[""4d596f261d4a8cfaaf6b8ced"", ""4b7d2213f964a520b5af2fe3"", … ""4c3efdc08d99b713a3021b06""]","[""swtcfe"", ""swtcuz"", … ""sxp75b""]","[""2012-12-09 12:22:54"", ""2013-02-12 19:00:02"", … ""2013-08-17 14:46:58""]",11
"[""4e385f6ee4cdedd4a1bb5f14"", ""4ba6c3d0f964a520466e39e3"", … ""4c1a83c5b306c928abba60b7""]","[""swg3cg"", ""swvswm"", … ""swg6ek""]","[""2012-04-14 14:06:36"", ""2013-03-09 02:00:53"", … ""2013-06-07 15:00:37""]",6
"[""4ac7abecf964a520fab820e3"", ""4ac7abecf964a520fab820e3""]","[""drkqd0"", ""drkqd0""]","[""2013-03-26 16:05:30"", ""2013-04-04 12:12:51""]",2
"[""4e2463e7b0fbdf9ba7c6d470"", ""4b6d43b6f964a520386e2ce3"", … ""4d603e079f67f04d7f8b76fb""]","[""sxk973"", ""sxk974"", … ""sxjdq3""]","[""2013-03-04 11:14:47"", ""2013-03-04 12:53:06"", … ""2013-08-10 21:51:45""]",14
"[""4ad4bff8f964a52011ea20e3"", ""4ad4bff8f964a52011ea20e3""]","[""dp4dme"", ""dp4dme""]","[""2012-05-26 22:46:34"", ""2012-05-27 12:31:48""]",2
…,…,…,…
"[""4ac518d2f964a52026a720e3"", ""4ac518d2f964a5203ca720e3"", … ""4b0f7b68f964a5209d6223e3""]","[""gcpvh0"", ""gcpvjf"", … ""u1hg6s""]","[""2012-06-01 15:55:06"", ""2012-06-03 13:18:40"", … ""2012-11-05 11:35:53""]",6
"[""4bb16585f964a520d3903ce3"", ""4bb16585f964a520d3903ce3"", … ""4adcda92f964a520f14b21e3""]","[""9q5fpx"", ""9q5fpx"", … ""u336yr""]","[""2012-11-01 21:29:59"", ""2012-11-12 23:04:41"", … ""2013-05-22 08:32:35""]",7
"[""4b2d0e1af964a52037cd24e3"", ""4b081ff6f964a520500423e3"", … ""4f106d06e4b0d59e1ee52310""]","[""6gycd7"", ""6gycfm"", … ""6gycc9""]","[""2012-04-17 08:10:12"", ""2012-04-17 10:09:40"", … ""2013-05-23 17:05:32""]",21


In [111]:
final_data_subset["times"][43].to_list(), final_sorted_sub["geohashes"][43].to_list()

(['2012-09-15 12:03:22',
  '2012-09-15 11:35:10',
  '2012-11-01 09:29:59',
  '2012-11-01 11:55:25',
  '2012-11-05 09:36:56',
  '2012-11-20 16:52:12'],
 ['9q8vzp', 'wy9vj3', 'dr5r80', 'drt3pb', 'dhwfxm', 'dr5r2p'])

In [113]:
final_sorted_sub["times_sorted"][43].to_list(), final_sorted_sub["geohashes"][
    43
].to_list()

(['2012-09-15 11:35:10',
  '2012-09-15 12:03:22',
  '2012-11-01 09:29:59',
  '2012-11-01 11:55:25',
  '2012-11-05 09:36:56',
  '2012-11-20 16:52:12'],
 ['9q8vzp', 'wy9vj3', 'dr5r80', 'drt3pb', 'dhwfxm', 'dr5r2p'])

In [74]:
def is_sorted_dates(dates):
    return dates == sorted(dates, key=to_UNIX_time)

In [84]:
for i in range(100):
    if not is_sorted_dates(final_data_subset["times"][i].to_list()):
        print(i)
        print(list(map(to_UNIX_time, final_data_subset["times"][i].to_list())))

43
[1347703402.0, 1347701710.0, 1351758599.0, 1351767325.0, 1352104616.0, 1353426732.0]


In [116]:
# import excel
import openpyxl

In [117]:
file = "dataset_TIST2015/elenco_studenti_training.xlsx"

In [118]:
wb = openpyxl.load_workbook(file)

In [122]:
import openpyxl


def reverse_names_in_excel(filename, sheet_name, column):
    # Load the Excel workbook
    wb = openpyxl.load_workbook(filename)

    # Select the active worksheet
    sheet = wb[sheet_name]

    # Iterate over the rows in the specified column, starting from the second row
    for row in range(1, sheet.max_row + 1):
        # Get the name from the specified column
        name = sheet[column + str(row)].value

        # Split the name into parts
        parts = name.split()

        # Reverse the parts and join them back together
        reversed_name = " ".join(parts[::-1])

        # Update the cell with the reversed name
        sheet[column + str(row)].value = reversed_name

    # Save the modified workbook
    wb.save(filename)


# Example usage:
filename = "dataset_TIST2015/elenco_studenti_training.xlsx"
reverse_names_in_excel(filename, "Foglio1", "A")